In [1]:
cd ..

/home/server3090ti/Projects/aesthetic-ai/DeepFL/DeepFL_binary


# check ToDos in def FindBestFeats

In [2]:
from final_utils import *
from sklearn.utils import shuffle

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


2022-11-03 11:34:40.950864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 11:34:40.951336: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 11:34:40.955146: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 11:34:40.955571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 11:34:40.956324: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [4]:
root_path = generate_root_path()
SC_CE_KLD = tf.keras.losses.SparseCategoricalCrossentropy()

In [5]:
def generate_init_pop(feat_vector_size, transformed_feat_vector_size, pop_size, init=True, init_size=3):
    # generates random population
    # INIT ONLY WHEN ORIGINAL DATA SHAPE! NOT DUMMY!
    k = transformed_feat_vector_size // pop_size
    path_list = glob(f'{root_path}/Projects/aesthetic-ai/DeepFL/DeepFL_binary/best_res/best_solution*')
    random.shuffle(path_list)
    population = []
    if init:
        for i in range(init_size):
            population.append(np.load(path_list[i]))
        
    len_pop = len(population)
    
    for i in range(1, pop_size - len_pop+1):
        main_pop = np.concatenate((np.ones(transformed_feat_vector_size - i*k), np.zeros(feat_vector_size - transformed_feat_vector_size + i*k)))
        np.random.shuffle(main_pop)
        population.append(main_pop)
        
    return np.array(population)

In [6]:
pop = generate_init_pop(19488, 5000, 8, init=True)

In [7]:
pop.sum(axis=1)

array([5000., 5000., 5000., 4375., 3750., 3125., 2500., 1875.])

In [8]:
def take_from_feats(data, pop):
    idx = np.nonzero(pop)
    new_data = np.squeeze(data[:, idx])
    return new_data

In [9]:
def load_for_current_bad(i=0):
    main_path=root_path + 'Data/AesthAI/alm/splitted/alm_train/' 
    feats_MG = 'original' 
    feats_CNN = 'border_600x600'
    cnn = 'cnn_efficientnet_b7'

    np.random.seed(i)
    alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
    bad_data = json.load(alm_train_bad)
    features_bad_list = []
    for data in bad_data:
        feat_path_1 = main_path + f'features/multigap/{feats_MG}/' + data['feature']
        feat_path_2 = main_path + f'features/{cnn}/{feats_CNN}/' + data['feature']
        connected = np.concatenate((np.squeeze(np.load(feat_path_1)), np.squeeze(np.load(feat_path_2))))
        features_bad_list.append(connected)
    features_bad_list = np.squeeze(np.array(features_bad_list))
    return features_bad_list
    

In [10]:
def loading_all_data_from_json(main_path = root_path + 'Data/AesthAI/alm/splitted/alm_train/', joint=False):
    features_good1 = []
    lbl = []
    data_all = []  
    feats_MG = 'original' 
    feats_CNN = 'border_600x600'
    cnn = 'cnn_efficientnet_b7'

    features_bad_list = list(map(load_for_current_bad,range(7)))

        
    alm_train_good = open(f'{main_path}/data_good1.json')
    good_data = json.load(alm_train_good)
    for data in good_data:
        feat_path_1 = main_path + f'features/multigap/{feats_MG}/' + data['feature']
        feat_path_2 = main_path + f'features/{cnn}/{feats_CNN}/' + data['feature']
        connected = np.concatenate((np.squeeze(np.load(feat_path_1)), np.squeeze(np.load(feat_path_2))))
        features_good1.append(connected)
    features_good1 = np.squeeze(np.array(features_good1))
    
    features_bad_list[0], features_bad1_val = extract_static_val_data(features_bad_list[0], perc = 0.11)
    features_good, features_good_val = extract_static_val_data(features_good1, perc = 0.11)
    
    
    val_data = np.concatenate( (features_bad1_val, features_good_val ) , axis=0 )
    val_lbl = np.concatenate( (np.zeros(len(features_bad1_val)), np.ones(len(features_good_val)) ), axis=0 )
    val_data, val_lbl = shuffle(val_data,val_lbl)
    
    for i in range(7):
        np.random.shuffle(features_good)
        data_i = np.concatenate( (features_good, features_bad_list[i] ) , axis=0 )
        lbl_i = np.concatenate( (np.ones(len(features_good)), np.zeros(len(features_bad_list[i])) ), axis=0 )
        data_i, lbl_i = shuffle(data_i,lbl_i)
        data_all.append(data_i)
        lbl.append(lbl_i)
        
    data_all = np.array(data_all, dtype=object)
    lbl = np.array(lbl, dtype=object)
    print(lbl[0].shape)
    
       
    if joint:
        clean_data = data_all[0]
        for i in range(1, 7):
            clean_data = np.vstack((clean_data, data_all[i]))

        clean_labels = np.array([lbl[0]]).T
        for i in range(1, len(lbl)):
            clean_labels = np.vstack((clean_labels, np.array([lbl[i]]).T))
  
    
        return clean_data, clean_labels, val_data, val_lbl
    return data_all, lbl, val_data, val_lbl
        

In [11]:
def selection( pop_acc, num_of_parents = 1):
    if num_of_parents > len(pop_acc)  :
        raise ValueError(f"Number of pairs must be less than  len(pop_acc)  ")
    sorted_parents = np.argsort(pop_acc)[: num_of_parents ]    #[:: -1]

    print(sorted_parents)
    return sorted_parents
    

In [12]:
# def crossover(pair, k=2):
#     parent1 = pair[0]
#     parent2 = pair[1]
#     print(parent1.shape)    
#     cut = parent1.shape[0] // (k + 1)
        
#     child1 = np.array(parent1)
#     child2 = np.array(parent2)
#     part1 = parent1[cut: k * cut]
#     part2 = parent2[cut: k * cut]

#     child1[cut: k * cut] = part2
#     child2[cut: k * cut] = part1

#     return child1, child2
# def crossover(pair, k=2):
#     parent1 = pair[0]
#     parent2 = pair[1]
#     cut = len(parent1) // (k + 1)
#     child1 = np.array(parent1)
#     child2 = np.array(parent2)
#     for i in range(k+1):
#         if i % 2 == 0: # ToDo: change and take randomly maybe
#             part1 = parent1[i * cut : (i + 1) * cut]
#             part2 = parent2[i * cut : (i + 1) * cut]
#             child1[i * cut : (i + 1) * cut] = part2
#             child2[i * cut : (i + 1) * cut] = part1

#     return child1, child2

# def crossover(pair, k=2):
#         parent1 = pair[0]
#         parent2 = pair[1]
#         size = parent1.sum()
#         cut = len(parent1) // (k + 1)
#         child1 = np.array(parent1, dtype=object)
#         child2 = np.array(parent2, dtype=object)
#         for i in range(k+1):
#             if i % 2 == 0: # ToDo: change and take randomly maybe
#                 part1 = parent1[i * cut : (i + 1) * cut]
#                 part2 = parent2[i * cut : (i + 1) * cut]
#                 child1[i * cut : (i + 1) * cut] = part2
#                 child2[i * cut : (i + 1) * cut] = part1
#         if child1.sum() != size:
#             change = child1.sum() - size
#             if change > 0:
#                 indx = np.nonzero(child1)[0]
#                 inx_to_change = np.random.choice(indx, size=int(abs(change)), replace=False)
#                 child1[inx_to_change] = 0 
#             else:
#                 indx = np.where(child1 == 0)[0]
#                 inx_to_change = np.random.choice(indx, size=int(abs(change)), replace=False)
#                 child1[inx_to_change] = 1
            
#         if child2.sum() != size:
#             change = child2.sum() - size
#             if change > 0:
#                 indx = np.nonzero(child2)[0]
#                 inx_to_change = np.random.choice(indx, size=int(abs(change)), replace=False)
#                 child2[inx_to_change] = 0 
#             else:
#                 indx = np.where(child2 == 0)[0]
#                 inx_to_change = np.random.choice(indx, size=int(abs(change)), replace=False)
#                 child2[inx_to_change] = 1   
# #         if np.unique(child1).shape[0] != child1.shape[0]:
# #             child1 = list(set(child1))
# #             repeats = np.delete(np.arange(19488), child1 )
# #             size = parent1.shape[0] - len(child1)
# #             add_elem = np.random.choice(repeats, size, False)
# #             child1.extend(add_elem)
# #             child1 = np.asarray(child1)

# #         if np.unique(child2).shape[0] != child2.shape[0]:
# #             child2 = list(set(child2))
# #             repeats = np.delete(np.arange(19488), child2 )
# #             size = parent2.shape[0] - len(child2)
# #             add_elem = np.random.choice(repeats, size, False)
# #             child2.extend(add_elem)
# #             child2 = np.asarray(child2)
        
#         return child1, child2
def crossover(matrix, k=2):
    sum_chromosome = np.sum(matrix[0])
    print(sum_chromosome)
    if matrix.shape[0] % 2 != 0:
        raise ValueError('Number of parents for crossover must be even!')
    matrix = matrix.reshape(int(matrix.shape[0]/2), 2, matrix.shape[1])
    tail_len = matrix.shape[2] % (k + 1)
    splitted_1 = np.split(matrix[:, 0, :matrix.shape[2]-tail_len], k+1, axis=1)
    splitted_1[len(splitted_1)-1] = np.concatenate((splitted_1[len(splitted_1)-1], matrix[:, 0, matrix.shape[2]-tail_len:]), axis=1)
    splitted_2 = np.split(matrix[:, 1, :matrix.shape[2]-tail_len], k+1, axis=1)
    splitted_2[len(splitted_2)-1] = np.concatenate((splitted_2[len(splitted_2)-1], matrix[:, 1, matrix.shape[2]-tail_len:]), axis=1)
    child_split_1 = splitted_1.copy()
    child_split_2 = splitted_2.copy()
    child_split_1[::2] = splitted_2[::2]
    child_split_2[::2] = splitted_1[::2]
    child_split_1 = np.concatenate(child_split_1, axis=1)
    child_split_2 = np.concatenate(child_split_2, axis=1)
    child_stack = np.stack((child_split_1, child_split_2), axis=2)
    child_concat_split = np.split(child_stack, child_stack.shape[0], axis=0)
    child_concat = np.concatenate(child_concat_split, axis=2)
    offspring = np.array(np.squeeze(child_concat.T))
    return offspring

In [13]:
def new_population(pop,pop_acc,children):
    indx_to_change = np.argsort(pop_acc)[:len(children)]
    pop[indx_to_change] = children
    return pop

In [14]:
def is_similar(population):
    pop_i = np.copy(population)
    idx_nan = np.where(pop_i == np.zeros(shape=(np.array(pop_i).shape[0], np.array(pop_i).shape[1])))
    pop_i[idx_nan[0], idx_nan[1]] = 5 # Hardcode
    is_sim = (population[None] == pop_i[:, None]).mean(axis=-1)
    idx = np.triu_indices(is_sim.shape[0])
    idx1 = idx[0]
    idx2 = idx[1]
    new_idx = np.where((idx1 == idx2) == False)
    idx1 = idx1[new_idx[0]]
    idx2 = idx2[new_idx[0]]
    idx = idx1, idx2
    sim_pop_idxs = np.where(is_sim[idx] >= 0.8) # 0.8 is threshold
    pop1 = idx1[sim_pop_idxs[0]]
    pop2 = idx2[sim_pop_idxs[0]]
    
    return pop1, pop2

def mutation_for_par(state, count_of_mut=100, similar_par=None):
    indx_0 = np.where(state == 0)[1]
    indx_1 = np.where(state == 1)[1]
    if similar_par is not None:
        idx_sim_1 = np.where(similar_par == 1)[1]
        conf_matrix = indx_0[:, None] == idx_sim_1[None]
        idx_to_del = np.where(conf_matrix)[0]
        indx_0 = np.delete(indx_0, idx_to_del)
        if len(indx_0) < count_of_mut:
            print('Not mutated')
            return state
    mutate_idxs_0 = np.random.choice(indx_0, size=count_of_mut, replace=False)
    mutate_idxs_1 = np.random.choice(indx_1, size=count_of_mut, replace=False)
    state[0, mutate_idxs_0] = 1
    state[0, mutate_idxs_1] = 0
    
    return state

def mutation(offspring):
    idx = is_similar(offspring)
    idx1 = np.unique(idx[0])
    idx2 = np.unique(idx[1])
    for idx_1, idx_2 in zip(idx1, idx2):
        offspring[idx_1] = mutation_for_par(state=(offspring[idx_1])[None], 
                                        count_of_mut=2, similar_par=(offspring[idx_2])[None])
    return offspring

In [15]:
def lr_exp_decay(epoch, lr):
    k = 0.048
    return lr * np.exp(-k*epoch)

def calc_acc_training(model, weights_path, X_test, y_test, batch_size):
    '''
    Compares Max classes with targets, getting mean class precision
    '''
    model.load_weights(weights_path)
    y_pred = model.predict(X_test, batch_size=batch_size, verbose=0)
    

    acc = (np.argmax(y_pred,axis=-1) == y_test).mean()
    
    
    return acc

def trainer(model, data, weights_path, data_val, batch_size=128, epochs=10, learning_rate=0.003):

    X_train, y_train = data

#     train_dataset = tf.data.Dataset.from_tensor_slices((np.array(X_train), np.array(y_train))).batch(batch_size)
    
    model.compile(loss=SC_CE_KLD,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, 
                                                  epsilon=1e-07, decay=0, amsgrad=False))
    model.load_weights(weights_path) 

    checkpoint = keras.callbacks.ModelCheckpoint(weights_path, 
                                                 monitor='val_loss', 
                                                 verbose=0, 
                                                 save_best_only=True, 
                                                 mode='min')
    
    
    schedule = tf.keras.callbacks.LearningRateScheduler(lr_exp_decay, verbose=0)
    
    callbacks_list = [checkpoint, schedule]
    with tf.device('/GPU:1'):
        history = model.fit(X_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=0,
                            callbacks=callbacks_list,
                            validation_data = data_val)

    return history

In [16]:
def training(data, data_val, model, weights_path, 
             batch_size, epochs, learning_rate):
#     # creating model, and weights
#     model = fc_model_softmax(input_num=(data[0][0].shape)[0]) #ToDo: check is all ok with shapes here --> checked
#     weights_path_pretrained = f'models/MultiGap_CNN_GA/custom_ga_popsize_10_vectorsize_5000.hdf5'#before training need to add weights path
#     model.save_weights(weights_path_pretrained) #if we want to cancel learning and start from 0
#     model.load_weights(weights_path_pretrained)
#     weights_path =  f'models/MultiGap_CNN_GA/custom_ga_popsize_10_vectorsize_5000.hdf5'
    
#     # choosing hyperparams
#     epochs = 15
#     batch_size = 128
#     learning_rate = 0.003
    
    # Training FC process
    history = trainer(model, data, weights_path, data_val, batch_size, epochs, 
                          learning_rate=learning_rate)
    acc = calc_acc_training(model, weights_path, data_val[0], data_val[1], batch_size)
    
#     print('----- Accuracy =', acc, ' -----')
    return acc

In [17]:
# ToDo: need to change these functions, there is no need pca, normalize, y --> checked
# we need y, because we use different resize functions for mg and cnn
def predict(x, y=None, model_gap=None, model=None, model_cnn=None, curr_pop=None):
    '''
    Does prediction on given numpy image using
    model_gap and model
    '''
    try:

        feat_mg = model_gap.predict(x, verbose=0)
    except:
        x = x[None]
        feat_mg = model_gap.predict(x, verbose=0)
        
    if model_cnn:
        feat_cnn = model_cnn.predict(y, verbose=0)
        feat = np.concatenate((np.squeeze(feat_mg), np.squeeze(feat_cnn)))
        feat = feat[None]
    else:
        feat = feat_mg
        
    if curr_pop is not None:
        feat = take_from_feats(feat, curr_pop)
        
    feat = feat[None]
    pred_score = model.predict(feat, verbose=0)

    return pred_score
    
def predict_from_path(model_gap, model, paths, curr_pop = None, resize_func=None, size=None, for_all=False, save_results=None, 
                      save_to=None, model_cnn=None):
    #always requires list of paths
    predicted = []
    
    for i, path in enumerate(paths):
        img_mg = read_img(path=path, resize_func=resize_func, size=size, for_all=for_all)
        img_cnn = None
        if model_cnn:
            img_cnn = read_img(path=path, resize_func=resize_add_border, size=(600, 600))
        pred_score = predict(img_mg, img_cnn, model_gap, model, model_cnn, curr_pop)
        predicted.append(pred_score)
    
    predicted = np.array(predicted)
    predicted = np.squeeze(predicted)
    
    if save_results:
        np.save(save_to, np.argmax(predicted, axis=-1))
        
    return predicted

def calc_acc_eval(labels, predicted):
    '''
    Calculating mean class error, e.g. predicted classes are 1vs0, 0vs0, 0vs0, 0vs0, then we have acc=0.25
    Inputs: 
        labels = target labels
        predicted = predicted binary probability distribution for the input
    Output:
        mean class error
    '''
    acc = np.sum(np.array(labels) == np.argmax(np.array(predicted), axis=1)) / len(labels)
    
    return np.round(acc * 100, 2)

In [18]:
def findBestFeats(pop_size, transformed_feat_vector_size, 
                  max_gen=50, num_of_parents=4, joint=False):

    good_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark', 'images', 'good', '*'))
    bad_imgs_path = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark', 'images', 'bad', '*'))

    good_imgs_path_2 = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark2', 'images', 'good', '*'))
    bad_imgs_path_2 = glob(os.path.join(root_path, 'Data', 'AesthAI', 'benchmark2', 'images', 'bad', '*'))
    
    paths_bench = good_imgs_path + bad_imgs_path
    labels_bench = np.concatenate((np.ones(len(good_imgs_path)), np.zeros(len(bad_imgs_path))))

    paths_bench_2 = good_imgs_path_2 + bad_imgs_path_2
    labels_bench_2 = np.concatenate((np.ones(len(good_imgs_path_2)), np.zeros(len(bad_imgs_path_2))))
    with tf.device('/GPU:1'):
        model_gap = model_mg = model_inceptionresnet_multigap()   
        model_cnn = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1",trainable=False) ])
    generation = 0
#     pop_acc = np.zeros(pop_size)
    data, lbl, val_data, val_lbl = loading_all_data_from_json(joint=joint)
    
    print('data loaded')
    
    def cycle_for_population(pop, model_gap=model_gap, paths_bench=paths_bench, paths_bench_2=paths_bench_2,
                        resize_max=resize_max, model_cnn=model_cnn, labels_bench=labels_bench, labels_bench2=labels_bench_2,
                        joint=joint, transformed_feat_vector_size = transformed_feat_vector_size):
        probs = [1,1,5]  
#         map(trainer_for_pair, range(7))
#         print('training finished')
        model = fc_model_softmax(input_num=int(pop.sum()))
        weights_path =  f'models/Softmax/MultiGap_CNN_GA/custom_ga_popsize_10_vectorsize_5000.hdf5' 
        model.save_weights(weights_path) #if we want to cancel learning and start from 0
        #ToDo: imast ka weighter sarqelu heto load anelu stex ? menak sarqely heriqa erevi!!! anunnel pretrained petq chi, bayc stugel nor anel

#         weights_path = f'models/Softmax/MultiGap_CNN_GA/custom_ga_popsize_10_vectorsize_5000.hdf5'  
#         model.load_weights(weights_path)
        
        # choosing hyperparams
        epochs = 15 # <---- be attentive
        batch_size = 128
        learning_rate = 0.003
        
        if joint:
#             print( "joint is true")
            curr_pop = pop
            data_transformed = take_from_feats(data, curr_pop)
            val_data_transformed = take_from_feats(val_data, curr_pop)
            acc_t = training((data_transformed, lbl), (val_data_transformed, val_lbl),
                                  model=model, weights_path=weights_path, batch_size=batch_size, 
                                 epochs=epochs, learning_rate=learning_rate)
            
        else:
#             print( "joint is false")
            def trainer_for_pair(i, data=data,curr_pop=pop, val_data=val_data, lbl=lbl, val_lbl=val_lbl):
                data_transformed = take_from_feats(data[i], curr_pop)
                val_data_transformed = take_from_feats(val_data, curr_pop)
                acc_t = training((data_transformed, lbl[i]), (val_data_transformed, val_lbl),
                                  model=model, weights_path=weights_path, batch_size=batch_size, 
                                 epochs=epochs, learning_rate=learning_rate)


            list(map(trainer_for_pair, range(7)))
            
#         model = fc_model_softmax(input_num=int(sum(pop))) #ToDo: imasty ka 2 angam model sarqelu u 2 angam weightery load anelu?
#         model.load_weights(weights_path)
        
        predicted1 = predict_from_path(model_gap=model_gap, model=model, paths=paths_bench, curr_pop=pop, resize_func=resize_max, 
                                  size=(996, 996), for_all=False, model_cnn=model_cnn)

        acc1 = calc_acc_eval(labels_bench, predicted1)
    #     print(f'Accuracy: {acc1} %')
    #     print('Done eval 1')

        predicted2 = predict_from_path(model_gap=model_gap, model=model, paths=paths_bench_2, curr_pop=pop, resize_func=resize_max, 
                                  size=(996, 996), for_all=False, model_cnn=model_cnn)

        acc2 = calc_acc_eval(labels_bench_2, predicted2)
        
        best = np.load('best_res/best.npy')
        acc1_best = best[0]
        acc2_best = best[1] 

        if acc1 > acc1_best and acc2 > acc2_best or (acc1+acc2)/2 > 90:
            np.save(f'best_res/best_solution_custom_{acc1}_{acc2}', indexes)
            model.save_weights(f'best_res/best_custom_{acc1}_{acc2}.hdf5')
#             acc1_best = acc1
#             acc2_best = acc2
#             best = np.array([acc1_best, acc2_best])
#             np.save('best_res/best', best)

    #     print(f'Accuracy: {acc1} %')
    #     print('Done eval 2')
        print(f'Acc bench1: {acc1}\nAcc bench2: {acc2}\nMean: {(acc1+acc2)/2}')
        fit_for_pop = probs[0] * (100 - acc1) + probs[1] * (100 - acc2) + probs[2] * pop.sum() / transformed_feat_vector_size
        print(f'fit_for_pop {fit_for_pop}')
        return    fit_for_pop #(acc1+acc2)/2
    
    if joint:
        pop = generate_init_pop(feat_vector_size=data.shape[1], 
                              transformed_feat_vector_size=transformed_feat_vector_size,
                              pop_size=pop_size)
        
    else:
        print(data[0].shape[1])
        pop = generate_init_pop(feat_vector_size=data[0].shape[1],
                              transformed_feat_vector_size=transformed_feat_vector_size,
                              pop_size=pop_size)
#     pop = np.asarray(pop)
    
    while generation < max_gen:
        pop_acc = list(map(cycle_for_population,pop)) 
        parent_indx = selection(pop_acc, num_of_parents=num_of_parents)
        parents = pop[parent_indx]  #np.array(pop[indx] for indx in parent_indx)
        print(parents)
        print(parents.shape)
#         children = []
#         for pair in parent_pairs:
#             children.extend(crossover((pop[pair[0]],pop[pair[1]])))
#         parents = np.squeeze(parents, axis=0)
        print(parents)
    
        children = crossover(parents)
        children_mutation = mutation(children)  
        pop = np.array(pop)
        pop = new_population(pop, pop_acc, children_mutation)
            
        #ToDo: selection function --> checked
        #ToDo: add crossover func --> checked
        #ToDo: add mutation func --> checked
        #ToDo: change population function
        
        
        generation += 1
        print(f'generation {generation}')
    return pop[np.argsort(pop_acc)[-1]]    

In [ ]:
best_feats = findBestFeats(pop_size = 12, transformed_feat_vector_size=5000, 
              max_gen=50, num_of_parents=8, joint=False)

(19364,)
data loaded
19488


2022-11-03 11:35:38.673487: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-11-03 11:36:49.862962: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-03 11:36:50.309192: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-03 11:36:50.310047: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-03 11:36:50.310060: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-03 11:36:50.310529: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-03 11:36:50.310576: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to

Acc bench1: 87.39
Acc bench2: 89.46
Mean: 88.425
fit_for_pop 28.150000000000006
Acc bench1: 85.59
Acc bench2: 88.97
Mean: 87.28
fit_for_pop 30.439999999999998
Acc bench1: 89.19
Acc bench2: 87.43
Mean: 88.31
fit_for_pop 28.379999999999995
Acc bench1: 79.28
Acc bench2: 84.53
Mean: 81.905
fit_for_pop 40.774
Acc bench1: 80.18
Acc bench2: 87.23
Mean: 83.70500000000001
fit_for_pop 36.75799999999999
Acc bench1: 84.68
Acc bench2: 87.91
Mean: 86.295
fit_for_pop 31.161999999999995
Acc bench1: 82.88
Acc bench2: 87.23
Mean: 85.055
fit_for_pop 33.226
Acc bench1: 75.68
Acc bench2: 85.78
Mean: 80.73
fit_for_pop 41.459999999999994
Acc bench1: 84.68
Acc bench2: 88.68
Mean: 86.68
fit_for_pop 29.143999999999988
Acc bench1: 86.49
Acc bench2: 87.52
Mean: 87.005
fit_for_pop 28.07800000000001
Acc bench1: 79.28
Acc bench2: 85.78
Mean: 82.53
fit_for_pop 36.611999999999995
